In [1]:
from pymongo import MongoClient
import os
import yaml
from pprint import pprint

In [2]:
import sys
import os
# Agregar el directorio 'src' al PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))

In [3]:
from utils.mongo_client import MongoClient
mongo = MongoClient()

2025-09-28 12:39:29.685 | INFO     | utils.mongo_client:_check_connection:68 - [MongoClient] Checking MongoDB connection...
2025-09-28 12:39:30.939 | INFO     | utils.mongo_client:_check_connection:70 - [MongoClient] MongoDB connection successful.
2025-09-28 12:39:30.941 | INFO     | utils.mongo_client:_verify_database_exists:34 - [MongoClient] Checking database existence...
2025-09-28 12:39:31.064 | INFO     | utils.mongo_client:_verify_database_exists:39 - [MongoClient] Database 'multimodal-rag' exists.
2025-09-28 12:39:31.067 | INFO     | utils.mongo_client:_verify_collection_exists:43 - [MongoClient] Checking collection existence...
2025-09-28 12:39:31.151 | INFO     | utils.mongo_client:_verify_collection_exists:48 - [MongoClient] Collection 'multimodal-rag' exists.
2025-09-28 12:39:31.152 | INFO     | utils.mongo_client:_verify_vector_index_exists:52 - [MongoClient] Checking vector index existence...
2025-09-28 12:39:31.365 | INFO     | utils.mongo_client:_verify_vector_index_exi

#### Vector Query

In [4]:
# Step -1: Create the Vector Index (Vector Search Model)
from pymongo.operations import SearchIndexModel

        {
        "fields":[
            {
            "type": "vector",
            "path": "<field-to-index>",
            "numDimensions": <number-of-dimensions>,
            "similarity": "euclidean | cosine | dotProduct",
            "quantization": "none | scalar | binary",
            "hnswOptions": {
                "maxEdges": <number-of-connected-neighbors>,
                "numEdgeCandidates": <number-of-nearest-neighbors>
            }
        ]
        }

## Vector Search Multimodal 

## Modelo de definición para un índice de Búsqueda Vectorial Multimodal en MongoDB Atlas
### Búsqueda Multimodal e Híbrida
Se crea un índice de búsqueda ultra-rápido sobre un campo específico para encontrar similitud de significado, sin importar si el dato original era texto o imagen.



### Configuración General
La línea type="vectorSearch" es fundamental. Le indica a MongoDB que este índice debe usar los nodos de búsqueda dedicados de Atlas Search, los cuales están optimizados para manejar datos vectoriales de alto rendimiento.

### Mapeo de Campos (mappings)
Dentro de la sección definition, se especifica exactamente cómo debe tratarse la información en tu colección:

- Indexación Dinámica ("dynamic": True): Esta directiva le permite a MongoDB indexar automáticamente todos los demás campos de tus documentos (como doc_path, image_path o type) que no están listados explícitamente. Esto es esencial para que puedas usar estos metadatos en la etapa de proyección ($project) más adelante.

- Campo de Búsqueda Vectorial (multimodal_embedding): Este es el corazón de tu estrategia.

Unificación: Se asume que este campo único contiene los embeddings generados por tu modelo multimodal (como CLIP) para ambos datos: los chunks de texto y las descripciones verbalizadas de las imágenes.

- Tipo ("knnVector"): Este tipo de dato activa el algoritmo de k-NN (Vecinos Más Cercanos), que organiza tus vectores en una estructura de grafo (como HNSW) para encontrar la similitud de significado de manera extremadamente rápida.

- Dimensiones ("dimensions": 768): Especifica que cada vector en este campo debe ser un array de 512 números. Es crucial que este valor coincida con la salida de tu modelo de embeddings.

- Métrica ("similarity": "cosine"): Indica que la cercanía entre los vectores se medirá usando la similitud de coseno, que es la métrica estándar para comparar la dirección de los embeddings y determinar su significado semántico.

#### En resumen:
Al ejecutar este código, le estás ordenando a MongoDB construir un mapa de significado de toda tu data. Una vez creado el índice, una sola consulta de texto podrá ser comparada con el campo multimodal_embedding y te devolverá los documentos de texto y las imágenes más relevantes de forma simultánea.

In [ ]:
search_multi_vector_model = SearchIndexModel(
    name="multi_vector_index",
    type="vectorSearch", # Añadir el tipo global del índice
    definition= {
        "mappings": { # <--- 'mappings' debe estar al nivel superior de 'definition'
            "dynamic": True,
            "fields": {
                "multimodal_embedding": {
                    "type": "knnVector",
                    "dimensions": 768, 
                    "similarity": "cosine" 
                }
            }
        }
    }
)

In [ ]:
mongo.collection.create_search_index(search_multi_vector_model)

In [ ]:
vectorsearch_multimodal_pipeline = [
  {
    "$vectorSearch": {
      "index": "multi_vector_index", 
      "path": "multimodal_embedding", 
      "queryVector": [
        
      ],
      "numCandidates": 20,
      "limit": 5
    }
  },
  {
    "$project": {
      "_id": 0, 
      "chunk_text": 1, 
      "doc_path": 1, 
      "image_path": 1, 
      "type": 1,
      "score": {
        "$meta": "vectorSearchScore"
      }
    }
  }
]

results = mongo.collection.aggregate(vectorsearch_multimodal_pipeline)
# print the results
pprint(list(results))